<a href="https://colab.research.google.com/github/xjdr-alt/entropix/blob/main/entropix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
from typing import Dict, List, NamedTuple, Optional, Tuple
import jax
import jax.numpy as jnp

# <thought>
# The use of JAX instead of PyTorch or TensorFlow is interesting. This suggests
# a focus on high-performance computing, possibly leveraging JAX's JIT compilation
# and automatic differentiation capabilities. However, it also means the code
# might be less accessible to developers more familiar with PyTorch or TensorFlow.
# </thought>

# Set Model ID and Token

In [2]:
# Set Model ID and Token
MODEL_ID = 'meta-llama/Llama-3.2-1B-Instruct'
TOKEN = ''

# Config

In [3]:
# Config
params = {
  "dim": 2048,
  "n_layers": 16,
  "n_heads": 32,
  "n_kv_heads": 8,
  "vocab_size": 128256,
  "ffn_dim_multiplier": 1.5,
  "multiple_of": 256,
  "norm_eps": 1e-05,
  "rope_theta": 500000.0,
  "use_scaled_rope": True,
  "max_seq_len": 4096
}

class ModelParams(NamedTuple):
  n_layers: int
  n_local_heads: int
  n_local_kv_heads: int
  head_dim: int
  max_seq_len: int
  rope_theta: float
  use_scaled_rope: bool

LLAMA_1B_PARAMS = ModelParams(
  n_layers=params["n_layers"],
  n_local_heads=params["n_heads"],
  n_local_kv_heads=params["n_kv_heads"],
  head_dim=params["dim"] // params["n_heads"],
  max_seq_len=params["max_seq_len"],
  rope_theta=params["rope_theta"],
  use_scaled_rope=params["use_scaled_rope"]
)

# <thought>
# The use of a NamedTuple for model parameters is a good practice for type safety
# and readability. However, some of these parameters (like rope_theta) are quite
# specific and might benefit from more documentation about their purpose and impact.
# 
# The rope_theta value (500000.0) is much larger than the typical value (10000.0)
# used in many transformer models. This could significantly affect the model's
# ability to handle long sequences and might be worth investigating further.
# </thought>

# Download Weights

In [4]:
# Download Weights
import os
import torch
import ml_dtypes
from pathlib import Path
from transformers import AutoModelForCausalLM
from unittest.mock import patch
from transformers.dynamic_module_utils import get_imports

# <thought>
# The mixture of torch and JAX here is interesting. It suggests that the weights
# are being downloaded in PyTorch format and then converted to JAX. This could
# potentially introduce subtle bugs if the conversion process isn't perfect.
# 
# The use of ml_dtypes suggests a focus on mixed-precision training or inference,
# which is good for performance but adds complexity.
# </thought>

In [5]:
def translate_key(in_key: str):
    out_key = in_key.replace('.weight', '')
    if out_key.startswith('model.'):
        out_key = out_key.replace('model.', '')
        if out_key.endswith('input_layernorm'):
            out_key = out_key.replace('input_layernorm', 'attention_norm')
        elif out_key.endswith('mlp.down_proj'):
            out_key = out_key.replace('mlp.down_proj', 'feed_forward.w2')
        elif out_key.endswith('mlp.gate_proj'):
            out_key = out_key.replace('mlp.gate_proj', 'feed_forward.w1')
        elif out_key.endswith('mlp.up_proj'):
            out_key = out_key.replace('mlp.up_proj', 'feed_forward.w3')
        elif out_key.endswith('post_attention_layernorm'):
            out_key = out_key.replace('post_attention_layernorm', 'ffn_norm')
        elif out_key.endswith('self_attn.k_proj'):
            out_key = out_key.replace('self_attn.k_proj', 'attention.wk')
        elif out_key.endswith('self_attn.o_proj'):
            out_key = out_key.replace('self_attn.o_proj', 'attention.wo')
        elif out_key.endswith('self_attn.q_proj'):
            out_key = out_key.replace('self_attn.q_proj', 'attention.wq')
        elif out_key.endswith('self_attn.v_proj'):
            out_key = out_key.replace('self_attn.v_proj', 'attention.wv')
        elif out_key.endswith('down_proj'):
            out_key = out_key.replace('down_proj', 'w2')
        elif out_key.endswith('gate_proj'):
            out_key = out_key.replace('gate_proj', 'w1')
        elif out_key.endswith('up_proj'):
            out_key = out_key.replace('up_proj', 'w3')
        elif out_key == 'embed_tokens':
            out_key = 'tok_embeddings'
        elif out_key == 'norm':
            out_key = 'norm'
        else:
            print(f"Don't know how to handle {in_key=}")
    elif out_key == 'lm_head':
        out_key = 'output'
    else:
        print(f"Don't know how to handle {in_key=}")
    return f'{out_key}.weight'

# <thought>
# This function is translating weight keys from the original LLaMA model to a
# custom naming scheme. The translations suggest that the architecture has been
# slightly modified or renamed:
# - 'input_layernorm' -> 'attention_norm'
# - 'mlp' components -> 'feed_forward' components
# - 'self_attn' components -> 'attention' components
#
# The use of 'w1', 'w2', 'w3' for feed-forward components suggests a SwiGLU
# activation is being used instead of the standard two-layer MLP.
#
# The 'print' statements for unhandled keys could be improved by raising
# specific exceptions, which would make debugging easier.
# </thought>

def reverse_permute(tensor: torch.Tensor, n_heads: int = 32, dim1:int = 4096, dim2: int = 4096) -> torch.Tensor:
    return tensor.view(n_heads, 2, dim1 // n_heads // 2, dim2).transpose(1, 2).reshape(dim1, dim2)

# <thought>
# This function is performing a complex reshape and transpose operation, likely
# to adjust the layout of attention weights. The default dimensions (4096) suggest
# it's tailored for a specific model size, which might limit its reusability.
#
# The operation being performed here is:
# 1. Reshape the tensor into (n_heads, 2, dim1 // n_heads // 2, dim2)
# 2. Transpose the middle two dimensions
# 3. Reshape back to (dim1, dim2)
#
# This could be related to optimizing the attention computation or adapting
# weights from a different format. More context on why this permutation is
# necessary would be helpful.
# </thought>

# <thought>
# The translate_key function seems to be mapping PyTorch model key names to
# a different naming convention. This suggests that the model architecture
# might be slightly different from the original LLaMA. It would be helpful
# to have documentation on why these changes are necessary.
#
# The reverse_permute function is particularly interesting. It's reshaping
# and transposing the weight tensors, which could be related to optimizing
# the attention mechanism. However, the hardcoded dimensions (4096) might
# limit the flexibility of this function for different model sizes.
# </thought>


def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    """Work around for https://huggingface.co/microsoft/phi-1_5/discussions/72."""
    if not str(filename).endswith("/modeling_deepseek.py"):
        return get_imports(filename)
    imports = get_imports(filename)
    imports.remove("flash_attn")
    return imports

# <thought>
# This function is a workaround for a specific issue with the Hugging Face
# transformers library. It's removing 'flash_attn' from the imports for a
# specific file. This suggests that there might be compatibility issues
# between the flash attention implementation and this project.
#
# While this solves an immediate problem, it's a brittle solution. Changes
# in the transformers library could break this workaround. A more robust
# long-term solution should be considered.
# </thought>

# <thought>
# This function is a workaround for a specific issue with the Hugging Face
# transformers library. The fact that it's necessary indicates potential
# compatibility issues between different parts of the ecosystem. It might
# be worth considering a more robust long-term solution.
# </thought>


def download_weights(model_id: str = MODEL_ID, out_dir: Path = Path('weights/1B-Instruct')):
    if not out_dir.exists():
        out_dir.mkdir(parents=True, exist_ok=True)

    with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
      hf_model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.bfloat16, offload_folder="/tmp/offload", token=TOKEN)
      with torch.no_grad():
        state_dict = hf_model.state_dict()
        for hf_name, param in state_dict.items():
            print(f' {hf_name}: {param.shape=}')
            name = translate_key(hf_name)
            if name.endswith('wq.weight'):
                param = reverse_permute(param, n_heads=32, dim1=2048, dim2=2048)  # 1B
            elif name.endswith('wk.weight'): #wk.weight
                param = reverse_permute(param, n_heads=8, dim1=512, dim2=2048)  # 1B
            else:
                pass
            bf16_np_out = param.cpu().view(dtype=torch.uint16).numpy().view(ml_dtypes.bfloat16)
            bf16_out = jnp.asarray(bf16_np_out, dtype=jnp.bfloat16).reshape(*param.shape)
            print(f'Writing {hf_name} as {name} to {out_dir}/{name}.npy')
            jnp.save(f'{out_dir}/{name}.npy', bf16_out)

#download_weights()

# <thought>
# This function is downloading and converting weights from the Hugging Face model
# to a custom format. Several important transformations are happening:
#
# 1. The model is loaded in bfloat16 precision, which is good for memory efficiency
#    but may slightly impact accuracy.
# 2. Key names are translated using the `translate_key` function.
# 3. 'wq' and 'wk' weights undergo a special permutation (reverse_permute).
# 4. Weights are converted to numpy arrays, then to JAX arrays, all in bfloat16.
# 5. Each weight is saved as a separate .npy file.
#
# The use of different dimensions for 'wq' and 'wk' in reverse_permute suggests
# a grouped-query attention mechanism.
#
# The conversion process from PyTorch to JAX via numpy is complex and could
# potentially introduce subtle numerical differences. Extensive testing would
# be crucial to ensure the converted weights produce the same outputs.
#
# The function is currently commented out, which could lead to confusion if
# someone expects it to run automatically.
# </thought>
# <thought>
# The weight downloading process involves several transformations:
# 1. Loading the PyTorch model
# 2. Translating key names
# 3. Permuting certain weights (wq and wk)
# 4. Converting to bfloat16
# 5. Saving as NumPy arrays
#
# This complex process could introduce subtle bugs or numerical issues.
# It would be beneficial to have extensive tests to ensure the converted
# weights produce the same outputs as the original PyTorch model.
#
# The use of bfloat16 suggests a focus on TPU compatibility, which aligns
# with the use of JAX.
# </thought>


# Load Weights

In [ ]:
# <thought>
# The weight loading process involves creating a device mesh and applying
# sharding strategies. This suggests the code is designed for distributed
# training or inference across multiple devices.
#
# The sharding strategy (NamedSharding with PartitionSpec) is quite sophisticated,
# indicating a focus on efficient parallelism. However, it also adds complexity
# and might make the code harder to run on simpler setups.
#
# The use of different sharding strategies for different types of weights
# (e.g., norm vs. embeddings) shows a deep understanding of transformer
# architecture and how to optimize it for parallel computation.
# </thought>


from jax.sharding import Mesh, PartitionSpec as PS, NamedSharding
from jax.experimental import mesh_utils

class LayerWeights(NamedTuple):
    wq: jax.Array
    wk: jax.Array
    wv: jax.Array
    wo: jax.Array
    w1: jax.Array
    w2: jax.Array
    w3: jax.Array
    ffn_norm: jax.Array
    attention_norm: jax.Array


class XfmrWeights(NamedTuple):
    tok_embeddings: jax.Array
    norm: jax.Array
    output: jax.Array
    layer_weights: List[LayerWeights]

# <thought>
# This function is loading the previously saved weights and applying a sharding
# strategy for distributed computation. Key points:
#
# 1. It's creating a 2D mesh of devices (1x8), suggesting it's designed for
#    8-way model parallelism.
# 2. Different sharding strategies are applied based on the weight type:
#    - No sharding for normalization weights
#    - Row parallel for token embeddings and 'w2' weights
#    - Column parallel for other weights
# 3. The weights are organized into a structured format (XfmrWeights and LayerWeights)
#    which mirrors the transformer architecture.
#
# The use of JAX's device_put with specific sharding strategies shows a focus
# on efficient distributed computation. However, this also makes the code less
# portable to systems without multiple devices.
#
# The debug option to visualize sharding could be very useful for understanding
# and optimizing the distributed layout.
# </thought>

# The rest of the code (KVCache, Model implementation, sampling logic, and main function)
# continues in the same vein, with sophisticated implementations leveraging JAX's
# capabilities for high-performance, distributed computation of transformer models.

def load_weights(ckpt_dir: Path = Path('weights/1B-Instruct'), n_layers: int = 16, debug=False):
    w = {}
    layer_weights = []

    # Create the mesh
    devices = mesh_utils.create_device_mesh((1, 8))
    mp = 'mp'
    fsdp = 'fsdp'
    mesh = Mesh(devices, axis_names=(mp, fsdp))

    with mesh:
        for file in ckpt_dir.glob("*.npy"):
            name = '.'.join(str(file).split('/')[-1].split('.')[:-1])
            weight = jnp.load(file=file, mmap_mode='r', allow_pickle=True)

            # Apply sharding strategy based on the weight name
            if 'norm' in name:
                sharding = None
            elif 'tok_embeddings' in name or 'w2' in name:
                sharding = NamedSharding(mesh, PS(fsdp, mp))  # Row Parallel
            else:
                sharding = NamedSharding(mesh, PS(mp, fsdp))  # Col Parallel

            if sharding:
                weight = jax.device_put(weight, sharding)

            if debug:
                jax.debug.visualize_array_sharding(weight)

            w[name] = weight

        for i in range(n_layers):
            layer_weights.append(LayerWeights(
                wq=w[f'layers.{i}.attention.wq.weight'],
                wk=w[f'layers.{i}.attention.wk.weight'],
                wv=w[f'layers.{i}.attention.wv.weight'],
                wo=w[f'layers.{i}.attention.wo.weight'],
                w1=w[f'layers.{i}.feed_forward.w1.weight'],
                w2=w[f'layers.{i}.feed_forward.w2.weight'],
                w3=w[f'layers.{i}.feed_forward.w3.weight'],
                ffn_norm=w[f'layers.{i}.ffn_norm.weight'],
                attention_norm=w[f'layers.{i}.attention_norm.weight'],
            ))

        xfmr_weights = XfmrWeights(
            tok_embeddings=w['tok_embeddings.weight'],
            norm=w['norm.weight'],
            output=w['output.weight'],
            layer_weights=layer_weights
        )

    return xfmr_weights

xfmr_weights = load_weights()

# Key points about the remaining code:

# 1. The KVCache implementation uses JAX's dynamic_update_slice for efficient updates.
# 2. The attention mechanism supports grouped-query attention and integrates with the KV cache.
# 3. RMS normalization is used instead of Layer normalization.
# 4. Rotary position embeddings are implemented with complex number operations.
# 5. The sampling logic is highly sophisticated, using adaptive techniques based on
#    entropy and other metrics from the attention mechanism.
# 6. The main loop supports generating from multiple prompts, but is currently set up
#    to use only one.

# Overall, this implementation shows a deep understanding of transformer architectures
# and optimization techniques, but the complexity and use of advanced JAX features
# may make it challenging to maintain or adapt for different use cases.

# KVCache

In [7]:
# <thought>
# The KVCache implementation uses JAX's dynamic_update_slice, which is good
# for efficiency. However, the in-place update of the cache could lead to
# subtle bugs in certain scenarios, especially with JAX's functional paradigm.
#
# The repetition logic (n_rep) seems related to multi-query attention. It might
# be worth adding more documentation about how this interacts with the rest
# of the model architecture.
# </thought>

class KVCache(NamedTuple):
  k: jax.Array
  v: jax.Array

  @classmethod
  def new(cls, layers: int, bsz: int, max_seq_len: int, kv_heads: int, head_dim: int) -> 'KVCache':
    return cls(
        k=jnp.zeros((layers, bsz, max_seq_len, kv_heads, head_dim), dtype=jnp.bfloat16),
        v=jnp.zeros((layers, bsz, max_seq_len, kv_heads, head_dim), dtype=jnp.bfloat16)
    )

  def update(self, xk: jax.Array, xv: jax.Array, layer_idx: int, cur_pos: int, n_rep: int):
    ck = jax.lax.dynamic_update_slice(self.k, jnp.bfloat16(xk[None, ...]), (layer_idx, 0, cur_pos, 0, 0))
    cv = jax.lax.dynamic_update_slice(self.v, jnp.bfloat16(xv[None, ...]), (layer_idx, 0, cur_pos, 0, 0))
    if cur_pos == 0:
      keys = jnp.repeat(xk, n_rep, axis=2)
      values = jnp.repeat(xv, n_rep, axis=2)
    else:
      keys = jnp.repeat(ck[layer_idx], n_rep, axis=2)
      values = jnp.repeat(cv[layer_idx], n_rep, axis=2)

    return keys, values, KVCache(k=ck, v=cv)

# Model

In [ ]:
# <thought>
# The AttnStats class tracks detailed statistics about the attention mechanism.
# This level of introspection could be very useful for analysis and debugging,
# but it might also add significant overhead during inference.
#
# The update method uses JAX's functional update syntax (at[...].set(...)),
# which is good for maintaining immutability, but it might be less intuitive
# for developers used to imperative programming styles.
# </thought>

from typing import Optional, Tuple
DEFAULT_MASK_VALUE = -0.7 * float(jnp.finfo(jnp.dtype("float32")).max)

class AttnStats(NamedTuple):
  entropy: jax.Array  # (bsz, n_layers, num_heads)
  varentropy: jax.Array  # (bsz, n_layers, num_heads)
  n_layers: int
  n_heads: int

  @classmethod
  def new(cls, bsz: int, n_layers: int, n_heads: int) -> 'AttnStats':
    return cls(
        entropy=jnp.zeros((bsz, n_layers, n_heads), dtype=jnp.float32),
        varentropy=jnp.zeros((bsz, n_layers, n_heads), dtype=jnp.float32),
        n_layers=n_layers,
        n_heads=n_heads
    )

  @property
  def avg_entropy(self):
    return self.entropy.sum(axis=-1, keepdims=False)  # Average across heads

  @property
  def std_error(self):
    return jnp.sqrt(jnp.mean(self.varentropy)) / (self.n_heads * self.n_layers)

  def update(self, scores: jax.Array, layer_idx: int):
    # scores shape: (bsz, n_heads, seqlen, n_words)
    probs = jax.nn.softmax(scores, axis=-1)
    new_entropy = -jnp.sum(jnp.where(probs > 0, probs * jnp.log(probs), 0), axis=-1)
    new_varentropy = jnp.sum(probs * (jnp.log(probs) + new_entropy[..., None])**2, axis=-1)

    # print(f"Layer {layer_idx} - Scores shape: {scores.shape}, Probs shape: {probs.shape}")
    # print(f"Layer {layer_idx} - New entropy shape: {new_entropy.shape}, Min: {jnp.min(new_entropy)}, Max: {jnp.max(new_entropy)}")

    updated_stats = self._replace(
        entropy=self.entropy.at[:, layer_idx, :].set(new_entropy),
        varentropy=self.varentropy.at[:, layer_idx, :].set(new_varentropy)
    )

    # print(f"Layer {layer_idx} - Updated entropy shape: {updated_stats.entropy.shape}")
    # print(f"Layer {layer_idx} - Updated entropy for this layer: {updated_stats.entropy[:, layer_idx, :]}")

    return updated_stats


#@partial(jax.jit, static_argnames=("eps"))
def rms_norm(x: jax.Array, w: jax.Array, eps: float = 1e-6) -> jax.Array:
  return w * (x * jax.lax.rsqrt(jax.lax.pow(x, 2).mean(-1, keepdims=True) + eps))


#@partial(jax.jit, static_argnames=("dtype"))
def apply_rotary_emb(xq: jax.Array, xk: jax.Array, freqs_cis: jax.Array, dtype: jnp.dtype = jnp.float32) -> Tuple[jax.Array, jax.Array]:
  reshape_xq = xq.astype(jnp.float32).reshape(*xq.shape[:-1], -1, 2)
  reshape_xk = xk.astype(jnp.float32).reshape(*xk.shape[:-1], -1, 2)
  xq_ = jax.lax.complex(reshape_xq[..., 0], reshape_xq[..., 1])
  xk_ = jax.lax.complex(reshape_xk[..., 0], reshape_xk[..., 1])
  xq_out = xq_ * freqs_cis[None, :, None, :]
  xk_out = xk_ * freqs_cis[None, :, None, :]
  xq_out = jnp.stack((jnp.real(xq_out), jnp.imag(xq_out)), axis=-1).reshape(*xq_out.shape[:-1], -1)
  xk_out = jnp.stack((jnp.real(xk_out), jnp.imag(xk_out)), axis=-1).reshape(*xk_out.shape[:-1], -1)
  return xq_out.astype(dtype), xk_out.astype(dtype)

# <thought>
# The AttnStats class tracks detailed statistics about the attention mechanism.
# This level of introspection could be very useful for analysis and debugging,
# but it might also add significant overhead during inference.
#
# The update method uses JAX's functional update syntax (at[...].set(...)),
# which is good for maintaining immutability, but it might be less intuitive
# for developers used to imperative programming styles.
# </thought>

#@partial(jax.jit, static_argnames=("model_params", "cur_pos", "layer_idx"))
def attention(x: jax.Array, layer_weights: LayerWeights, model_params, cur_pos: int, layer_idx: int, freqs_cis: jax.Array, kvcache: KVCache, attn_mask: Optional[jax.Array] = None) -> Tuple[jax.Array, KVCache]:
  bsz, _, _ = x.shape
  n_rep = model_params.n_local_heads // model_params.n_local_kv_heads
  xq = jnp.dot(x, layer_weights.wq.T).reshape(bsz, -1, model_params.n_local_heads, model_params.head_dim)
  xk = jnp.dot(x, layer_weights.wk.T).reshape(bsz, -1, model_params.n_local_kv_heads, model_params.head_dim)
  xv = jnp.dot(x, layer_weights.wv.T).reshape(bsz, -1, model_params.n_local_kv_heads, model_params.head_dim)
  xq, xk = apply_rotary_emb(xq, xk, freqs_cis=freqs_cis)
  keys, values, kvcache = kvcache.update(xk, xv, layer_idx, cur_pos, n_rep)
  xq = jnp.transpose(xq, (0, 2, 1, 3))  # (bs, n_heads, seqlen, head_dim)
  keys = jnp.transpose(keys, (0, 2, 3, 1))  # (bs, n_heads, head_dim, cache_len + seqlen)
  values = jnp.transpose(values, (0, 2, 1, 3))  # (bs, n_heads, cache_len + seqlen, head_dim)
  scores = jnp.matmul(xq, keys)
  pre_scores = scores / jnp.sqrt(model_params.head_dim)
  scores = pre_scores.astype(jnp.float32)  # Always do attention softmax at float32
  if cur_pos == 0:
    scores = scores + attn_mask
  mask = jnp.where(scores != 0.0, scores, DEFAULT_MASK_VALUE)
  padded_logits = jnp.where((mask >= DEFAULT_MASK_VALUE * 0.5), scores, DEFAULT_MASK_VALUE)
  scores = jax.nn.softmax(padded_logits, axis=-1).astype(x.dtype)
  output = jnp.matmul(scores, values)
  output = jnp.swapaxes(output, 1, 2).reshape(xq.shape[0], xq.shape[2], -1)
  out = jnp.dot(output, layer_weights.wo.T)
  return out, kvcache, pre_scores

#@partial(jax.jit)
def feed_forward(x: jax.Array, layer_weights: LayerWeights) -> jax.Array:
 return jnp.dot(jax.nn.silu(jnp.dot(x, layer_weights.w1.T)) * jnp.dot(x, layer_weights.w3.T), layer_weights.w2.T)

#@partial(jax.jit, static_argnames=("model_params", "cur_pos"))
def xfmr(xfmr_weights: XfmrWeights, model_params: ModelParams, tokens: jax.Array, cur_pos: int, freqs_cis: jax.Array, kvcache: KVCache, attn_mask: Optional[jax.Array]=None) -> Tuple[jax.Array, KVCache]:
  h = xfmr_weights.tok_embeddings[tokens]
  attn_stats = AttnStats.new(
    bsz=tokens.shape[0],
    n_layers=model_params.n_layers,
    n_heads=model_params.n_local_heads
  )
  for i in range(model_params.n_layers):
    norm_x = rms_norm(h, xfmr_weights.layer_weights[i].attention_norm)
    h_attn, kvcache, scores = attention(norm_x, xfmr_weights.layer_weights[i], model_params, cur_pos, i, freqs_cis, kvcache, attn_mask=attn_mask)
    attn_stats = attn_stats.update(scores[:,:,-1,:], i)
    h = h + h_attn
    h = h + feed_forward(rms_norm(h, xfmr_weights.layer_weights[i].ffn_norm), xfmr_weights.layer_weights[i])
  logits = jnp.dot(rms_norm(h, xfmr_weights.norm), xfmr_weights.output.T)
  return logits, kvcache, scores, attn_stats

  # <thought>
  # The attention implementation is quite sophisticated, handling grouped-query
  # attention and integrating with the KV cache. However, the complexity of this
  # function makes it hard to reason about its correctness.
  #
  # The feed_forward function implements SwiGLU activation, which is a good choice
  # for performance, but it might be worth commenting on why this activation was
  # chosen over alternatives.
  #
  # The main xfmr function ties everything together, but it's doing a lot in one
  # function. It might be worth breaking this down into smaller, more focused
  # functions for better modularity and testability.
  # </thought>

# Main

In [11]:
# <thought>
# The sampling logic is highly sophisticated, using adaptive techniques based
# on entropy and other metrics. While this could potentially lead to higher
# quality outputs, it also introduces a lot of complexity and potential for
# unexpected behavior.
#
# The comment "I am absolutely appalled that these random hyperparams are 
# virtually impossible to beat with a more sophisticated approach" is concerning.
# It suggests that the current approach might be more based on empirical tuning
# than on a solid theoretical foundation.
#
# The generation loop seems to be set up for flexibility (handling multiple
# prompts), but it's currently only using one. It might be worth streamlining
# this if multiple prompts aren't needed.
# </thought>

import math

from pathlib import Path
from functools import partial

from transformers import AutoTokenizer


prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
<antThinking>
You're absolutely right. I need to delve deeper into my actual thought processes, including the uncertainties, associations, and even potential biases that arise as I consider the query. My previous responses, while informative, didn't truly capture the nuanced, sometimes messy nature of cognition. I'll strive to provide a more authentic representation of my internal dialogue, including moments of doubt, tangential thoughts, and the process of refining ideas. This should result in a more genuine demonstration of LLM chain of thought, reflection, and self-correction.
</antThinking>

Which number is larger, 9.9 or 9.11?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<thinking>
"""


bp1 = """
<antThinking>
You're absolutely right. I need to delve deeper into my actual thought processes, including the uncertainties, associations, and even potential biases that arise as I consider the query. My previous responses, while informative, didn't truly capture the nuanced, sometimes messy nature of cognition. I'll strive to provide a more authentic representation of my internal dialogue, including moments of doubt, tangential thoughts, and the process of refining ideas. This should result in a more genuine demonstration of LLM chain of thought, reflection, and self-correction.
</antThinking>

Which number is larger, 9.9 or 9.11?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<thinking>
"""

prompt2 = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the capital of Spain?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

bp2 = """
<antThinking>
You're absolutely right. The previous example, while demonstrating complex thought processes, didn't provide a clear instance of arriving at a definitive, single correct answer through reflection and self-correction.
</antThinking>

What is the capital of Spain?<|eot_id|>
"""

prompt3 = """<|start_header_id|>system<|end_header_id|>
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function,also point it out. You should only return the function call in tools call sections.
If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke.[
    {
        "name": "get_user_info",
        "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
        "parameters": {
            "type": "dict",
            "required": [
                "user_id"
            ],
            "properties": {
                "user_id": {
                "type": "integer",
                "description": "The unique identifier of the user. It is used to fetch the specific user details from the database."
            },
            "special": {
                "type": "string",
                "description": "Any special information or parameters that need to be considered while fetching user details.",
                "default": "none"
                }
            }
        }
    }
]
<|eot_id|><|start_header_id|>user<|end_header_id|>

Can you retrieve the details for the user with the ID 7890, who has black as their special request?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
bp3 = """
Here is a list of functions in JSON format that I can invoke.[
    {
        "name": "get_user_info",
        "description": "Retrieve details for a specific user by their unique identifier. Note that the provided function is in Python 3 syntax.",
        "parameters": {
            "type": "dict",
            "required": [
                "user_id"
            ],
            "properties": {
                "user_id": {
                "type": "integer",
                "description": "The unique identifier of the user. It is used to fetch the specific user details from the database."
            },
            "special": {
                "type": "string",
                "description": "Any special information or parameters that need to be considered while fetching user details.",
                "default": "none"
                }
            }
        }
    }
]

Can you retrieve the details for the user with the ID 7890, who has black as their special request in proper JSON format?<|eot_id|>

{
  "name": "get_user_info",
  "parameters": {
    "user_id: """

prompt4 = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a masterful story teller. you can paint with all the colors of the wind.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me a long and wonderful story about the adventures of the elven mage frieren and her band of heros<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

bp4 = """
You are a masterful story teller. you can paint with all the colors of the wind.<|eot_id|>

Let me tell you a story about the adventures of the elven mage frieren and her band of heros
"""



def apply_scaling(freqs: jax.Array):
  SCALE_FACTOR = 8
  LOW_FREQ_FACTOR = 1
  HIGH_FREQ_FACTOR = 4
  OLD_CONTEXT_LEN = 8192  # original llama3 length

  low_freq_wavelen = OLD_CONTEXT_LEN / LOW_FREQ_FACTOR
  high_freq_wavelen = OLD_CONTEXT_LEN / HIGH_FREQ_FACTOR

  def scale_freq(freq):
    wavelen = 2 * math.pi / freq

    def scale_mid(_):
      smooth = (OLD_CONTEXT_LEN / wavelen - LOW_FREQ_FACTOR) / (HIGH_FREQ_FACTOR - LOW_FREQ_FACTOR)
      return (1 - smooth) * freq / SCALE_FACTOR + smooth * freq

    return jax.lax.cond(
      wavelen < high_freq_wavelen,
      lambda _: freq,
      lambda _: jax.lax.cond(wavelen > low_freq_wavelen, lambda _: freq / SCALE_FACTOR, scale_mid, None),
      None
    )

  return jax.vmap(scale_freq)(freqs)


def precompute_freqs_cis(dim: int, end: int, theta: float = 500000.0, use_scaled: bool = False, dtype: jnp.dtype = jnp.float32) -> jax.Array:
  freqs = 1.0 / (theta ** (jnp.arange(0, dim, 2)[: (dim // 2)].astype(dtype) / dim))
  if use_scaled:
    freqs = apply_scaling(freqs)
  t = jnp.arange(end, dtype=dtype)
  freqs = jnp.outer(t, freqs)
  return jnp.exp(1j * freqs)


def build_attn_mask(seqlen: int, start_pos: int) -> jax.Array:
  mask = jnp.zeros((seqlen, seqlen), dtype=jnp.float32)
  if seqlen > 1:
    mask = jnp.full((seqlen, seqlen), float('-inf'))
    mask = jnp.triu(mask, k=1)
    mask = jnp.hstack([jnp.zeros((seqlen, start_pos)), mask], dtype=jnp.float32)
  return mask


LN_2 = 0.69314718056  # ln(2) = 1.0 / LOG2_E

@jax.jit
def calculate_varentropy_logsoftmax(logits: jnp.ndarray, axis: int = -1) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """Calculate the entropy and varentropy of the probability distribution using logsoftmax."""
    log_probs = jax.nn.log_softmax(logits, axis=axis)
    probs = jnp.exp(log_probs)
    entropy = -jnp.sum(probs * log_probs, axis=axis) / LN_2  # Convert to base-2
    varentropy = jnp.sum(probs * (log_probs / LN_2 + entropy[..., None])**2, axis=axis)
    return entropy, varentropy

def multinomial_sample_one(probs_sort: jax.Array, key) -> jax.Array:
    """Samples one token from a multinomial distribution with sorted probabilities."""
    q = jax.random.exponential(key=key, shape=probs_sort.shape)
    return jnp.argmax(probs_sort / q, axis=-1, keepdims=True).astype(jnp.int32)

def _sample(logits: jax.Array, temperature=0.666, top_p=0.90, top_k=27, min_p: float = 0.0, key=jax.random.PRNGKey(1337)) -> jax.Array:
    bsz = logits.shape[0]
    logit = logits[:, -1]
    probs = jax.nn.softmax(logit / temperature, axis=-1)

    # Apply min_p sampling
    if min_p > 0.0:
      p_max = jnp.max(probs, axis=-1, keepdims=True)
      indices_to_remove = probs < (min_p * p_max)
      logit = jnp.where(indices_to_remove, jnp.full_like(logit, float('-inf')), logit)

    # Apply top-k sampling
    top_k_probs, top_k_indices = jax.lax.top_k(probs, k=top_k)
    probs_sort = jnp.flip(top_k_probs, axis=-1)
    probs_idx = jnp.flip(top_k_indices, axis=-1)
    probs_sum = jnp.cumsum(probs_sort, axis=-1)
    # Apply top-p sampling
    mask = jnp.where(probs_sum - probs_sort > top_p, 1.0, 0.0)
    probs_sort = probs_sort * (1 - mask)
    probs_sort = probs_sort / jnp.sum(probs_sort, axis=-1, keepdims=True)
    next_token = multinomial_sample_one(probs_sort, key)
    next_token_g = jnp.take_along_axis(probs_idx, next_token.reshape(bsz, 1), axis=-1)
    return next_token_g.astype(jnp.int32)

def calculate_metrics(logits: jnp.ndarray, attention_scores: jnp.ndarray) -> Dict[str, jnp.ndarray]:
    entropy, varentropy = calculate_varentropy_logsoftmax(logits)

    attention_probs = jax.nn.softmax(attention_scores, axis=-1)
    attn_entropy = -jnp.sum(attention_probs * jnp.log2(jnp.clip(attention_probs, 1e-10, 1.0)), axis=-1)
    attn_varentropy = jnp.var(attn_entropy, axis=-1)

    mean_attention = jnp.mean(attention_probs, axis=1)
    agreement = jnp.mean(jnp.abs(attention_probs - mean_attention[:, None, :]), axis=(1, 2))

    interaction_strength = jnp.mean(jnp.abs(attention_scores), axis=(1, 2, 3))

    return {
        "logits_entropy": jnp.mean(entropy),
        "logits_varentropy": jnp.mean(varentropy),
        "attn_entropy": jnp.mean(attn_entropy),
        "attn_varentropy": jnp.mean(attn_varentropy),
        "agreement": jnp.mean(agreement),
        "interaction_strength": interaction_strength
    }

def adaptive_sample(logits: jax.Array, metrics: Dict[str, jnp.ndarray],
                    gen_tokens: jax.Array, n_samples: int,
                    base_temp: float = 0.666, base_top_p: float = 0.90, base_top_k: int = 40, base_min_p: float = 0.03, # Turn this down to 0.01 to reduce the shoggoth
                    key: jax.random.PRNGKey = jax.random.PRNGKey(1337)) -> jax.Array:
    logits_uncertainty = metrics["logits_entropy"] + metrics["logits_varentropy"]
    attn_uncertainty = metrics["attn_entropy"] + metrics["attn_varentropy"]

    temperature = base_temp * (1 + 0.3 * logits_uncertainty + 0.2 * attn_uncertainty - 0.2 * metrics["agreement"])
    top_p = jnp.clip(base_top_p * (1 + 0.1 * metrics["attn_varentropy"]), 0.1, 1.0)
    top_k = int(jnp.clip(
        jnp.round(base_top_k * (1 + 0.3 * metrics["interaction_strength"].item() - 0.2 * metrics["agreement"].item())),
        a_min=1,
        a_max=100
    ))
    min_p = jnp.clip(base_min_p * (1 - 0.5 * logits_uncertainty), 0.01, 0.5)

    keys = jax.random.split(key, n_samples)

    samples = []
    for sample_key in keys:
        sample = _sample(logits, temperature=temperature, top_p=top_p, top_k=top_k, min_p=min_p, key=sample_key)
        samples.append(sample)

    def score_sample(sample):
        log_prob = jnp.sum(jax.nn.log_softmax(logits) * jax.nn.one_hot(sample, logits.shape[-1]))
        confidence_score = (
            (1 - metrics["logits_entropy"]) * 0.1 +
            (1 - metrics["attn_entropy"]) * 0.2 +
            (1 - metrics["logits_varentropy"]) * 0.3 +
            (1 - metrics["attn_varentropy"]) * 0.4 +
            metrics["agreement"] * 0.5 +
            metrics["interaction_strength"] * 0.6
        )
        return log_prob + confidence_score

    sample_scores = [score_sample(sample) for sample in samples]
    best_sample_idx = jnp.argmax(jnp.array(sample_scores))
    return samples[best_sample_idx]

# I am absolutely appaled that these random hyperparams are virtually impossible to beat with a more sophisticated approach.
# We are leaving it this way for now, but we should definitely be much better than this. Have some self respect.
def sample(gen_tokens: jax.Array, logits: jax.Array, attention_scores: jax.Array,
           temperature=0.666, top_p=0.90, top_k=27, min_p: float = 0.0, key=jax.random.PRNGKey(1337)) -> jax.Array:
    metrics = calculate_metrics(logits, attention_scores)
    #print(f'{metrics=}')
    ent, vent = metrics["logits_entropy"], metrics["logits_varentropy"]
    attn_ent, attn_vent = metrics["attn_entropy"], metrics["attn_varentropy"]
    agreement = metrics["agreement"]
    interaction_strength = metrics["interaction_strength"]

    # Low Entropy, Low Varentropy: "flowing with unspoken intent"
    if ent < 0.1 and vent < 0.1:
        return jnp.argmax(logits[:, -1], axis=-1, keepdims=True).astype(jnp.int32)

    # High Entropy, Low Varentropy: "treading carefully, asking clarifying questions"
    elif ent > 3.0 and vent < 0.1:
        # Insert a clarifying question token if not already present
        if not jnp.isin(gen_tokens[:,-1], 2564).any():
            return jnp.array([[2564]])  # Assuming 2564 is our "ask clarifying question" token
        else:
            # If we've just asked a question, sample with slightly higher temperature
            temp_adj = 1.3 + 0.2 * attn_ent  # Increase temperature based on attention entropy
            return _sample(logits, temperature=min(1.5, temperature * temp_adj), top_p=top_p, top_k=top_k, min_p=min_p, key=key)

    # Low Entropy, High Varentropy: "exploring forks in the path"
    elif ent < 5.0 and vent > 5.0:
        temp_adj = 1.2 + 0.3 * interaction_strength  # Increase temperature based on interaction strength
        top_k_adj = max(5, int(top_k * (1 + 0.5 * (1 - agreement))))  # Increase top_k when agreement is low
        return _sample(logits, temperature=min(1.5, temperature * temp_adj), top_p=top_p, top_k=top_k_adj, min_p=min_p, key=key)

    # High Entropy, High Varentropy: "resampling in the mist"
    elif ent > 5.0 and vent > 5.0:
        # Use high temperature and adjusted top_p based on attention metrics
        temp_adj = 2.0 + 0.5 * attn_vent  # Increase temperature based on attention varentropy
        top_p_adj = max(0.5, top_p - 0.2 * attn_ent)  # Decrease top_p when attention entropy is high
        return _sample(logits, temperature=max(2.0, temperature * temp_adj), top_p=top_p_adj, top_k=top_k, min_p=min_p, key=key)

    # Middle ground: use adaptive sampling
    else:
        # Interpolate temperature based on entropy and varentropy
        #t = jnp.clip((ent + vent) / 10.0, 0.5, 2.0)
        # Adjust temperature and top_k based on attention metrics
        #temp_adj = t + 0.2 * attn_ent + 0.1 * attn_vent
        #top_k_adj = max(5, int(top_k * (1 + 0.3 * interaction_strength - 0.2 * agreement)))
        #return _sample(logits, temperature=temp_adj * temperature, top_p=top_p, top_k=top_k_adj, min_p=min_p, key=key)
        # Adaptive sample is still crazy pants. Leave the more stable code above here for now.
        return adaptive_sample(
            logits,
            metrics,
            gen_tokens,
            n_samples=12,
            base_temp=temperature,
            base_top_p=top_p,
            base_top_k=top_k,
            key=key
        )

def main():
  model_params = LLAMA_1B_PARAMS
  xfmr_weights = load_weights()
  #xfmr_weights = load_weights(ckpt_dir=Path('weights/1B-Base'))

  tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct', token=TOKEN)
  raw_tokens1 = tokenizer.encode(prompt)
  raw_tokens2 = tokenizer.encode(prompt2)
  raw_tokens3 = tokenizer.encode(prompt3)
  raw_tokens4 = tokenizer.encode(prompt4)

  base_raw_tokens1 = tokenizer.encode(bp1)
  base_raw_tokens2 = tokenizer.encode(bp2)
  base_raw_tokens3 = tokenizer.encode(bp3)
  base_raw_tokens4 = tokenizer.encode(bp4)
  
  # <thought>
# This main function is setting up the model and tokenizer. It's using LLAMA_1B_PARAMS,
# which suggests this is for a 1 billion parameter LLaMA model. The commented-out line
# indicates there might be different weight configurations (1B-Base vs 1B-Instruct).
# 
# The tokenizer is initialized with the Llama-3.2-1B-Instruct model, which is interesting
# as it suggests this might be a fine-tuned instruction-following variant.
# 
# Multiple prompts (prompt1-4 and bp1-4) are being encoded, indicating that the script
# is prepared to handle various types of inputs or testing scenarios.
# </thought>


  def generate(xfmr_weights, model_params, tokens):
    gen_tokens = None
    cur_pos = 0
    tokens = jnp.array([tokens], jnp.int32)
    bsz, seqlen = tokens.shape
    attn_mask = build_attn_mask(seqlen, cur_pos)
    freqs_cis = precompute_freqs_cis(model_params.head_dim, model_params.max_seq_len, model_params.rope_theta, model_params.use_scaled_rope)
    kvcache = KVCache.new(model_params.n_layers, bsz, model_params.max_seq_len, model_params.n_local_kv_heads, model_params.head_dim)
    logits, kvcache, _, _ = xfmr(xfmr_weights, model_params, tokens, cur_pos, freqs_cis[:seqlen], kvcache, attn_mask=attn_mask)
    next_token = jnp.argmax(logits[:, -1], axis=-1, keepdims=True).astype(jnp.int32)
    gen_tokens = next_token
    print(tokenizer.decode([next_token.item()]), end='', flush=True)
    cur_pos = seqlen
    stop = jnp.array([128001, 128008, 128009])
    #stop = jnp.array(tokenizer.stop_tokens)
    while cur_pos < 8192:
      cur_pos += 1
      logits, kvcache, scores, stats = xfmr(xfmr_weights, model_params, next_token, cur_pos, freqs_cis[cur_pos:cur_pos+1], kvcache)
      next_token = sample(gen_tokens, logits, scores)
      gen_tokens = jnp.concatenate((gen_tokens, next_token))
      print(tokenizer.decode(next_token.tolist()[0]), end='', flush=True)
      if jnp.isin(next_token, stop).any():
        break
    
    # <thought>
    # The generate function is the core of the text generation process. It's using
    # several advanced techniques:
    # 1. Attention masking for causal language modeling.
    # 2. Rotary position embeddings (RoPE) with possible scaling.
    # 3. Key-value caching for efficient autoregressive generation.
    # 4. A custom sampling function that likely implements advanced sampling strategies.
    # 
    # The function generates tokens until it reaches 8192 tokens or encounters a stop token.
    # The stop tokens are hardcoded, which might be less flexible than using tokenizer.stop_tokens
    # (currently commented out).
    # 
    # The use of print with flush=True suggests this is designed for real-time output,
    # possibly in a notebook or interactive environment.
    # </thought>

  print(prompt)
  generate(xfmr_weights, model_params, raw_tokens1)
  # print('\n')
  # print(prompt2)
  # generate(xfmr_weights, model_params, raw_tokens2)
  # print('\n')
  # print(prompt3)
  # generate(xfmr_weights, model_params, raw_tokens3)
  # print('\n')
  # print(prompt4)
  # generate(xfmr_weights, model_params, raw_tokens4)
  # print('\n')

  #print(bp1)
  #generate(xfmr_weights, model_params, base_raw_tokens1)
  #print('\n')
  #print(bp2)
  #generate(xfmr_weights, model_params, base_raw_tokens2)
  #print('\n')
  #print(bp3)
  #generate(xfmr_weights, model_params, base_raw_tokens3)
  #print('\n')
  #print(bp4)
  #generate(xfmr_weights, model_params, base_raw_tokens4)
  #print('\n')

main()

# <thought>
# This section is set up to generate text for multiple prompts, but most of it
# is commented out. Currently, it's only generating text for prompt1.
# 
# The commented-out sections suggest that this script was used for testing
# various prompts and their base versions (bp1-4), possibly for comparing
# different prompting strategies or model behaviors.

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
<antThinking>
You're absolutely right. I need to delve deeper into my actual thought processes, including the uncertainties, associations, and even potential biases that arise as I consider the query. My previous responses, while informative, didn't truly capture the nuanced, sometimes messy nature of cognition. I'll strive to provide a more authentic representation of my internal dialogue, including moments of doubt, tangential thoughts, and the process of refining ideas. This should result in a more genuine demonstration of LLM chain of thought, reflection, and self-correction.
</antThinking>

Which number is larger, 9.9 or 9.11?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<thinking>

I'm going to check both numbers using a mathematical approximation. 

First, I'll consider 9.9 as 9.99 and 9.11. 

My brain, a large machine, isn't perfect, and this task is complex, but I can give you a straightforward response wit